In [ ]:
# Import packages 
import os
import sys

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.tri as tri
import matplotlib.colors as colors

from scipy.io import FortranFile

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import importlib
import glob
import copy

import Plots as Po

importlib.reload(Po)

#sys.path.append('../Plotting/')
#   Now you can imprt modules in ../Plotting
#import xyp_plot as xyp
#import ana as a


In [ ]:
host=os.getenv('HOST')
host

In [ ]:
#---------------------------------------------------------------
# Function to perform quick coarse graining in lieu of more 
# more accurate conservative remapping
#---------------------------------------------------------------
def coarse_grain(array, block_size, lsum=False):
    # Get the shape of the input array
    m, n = array.shape

    lmean=(not lsum)
    
    # Ensure the input array can be evenly divided into blocks
    assert m % block_size == 0 and n % block_size == 0, "Array dimensions must be divisible by the block size"
    
    # Reshape the array to a 4D array (blocks)
    reshaped_array = array.reshape(m // block_size, block_size, n // block_size, block_size)
    
    if (lmean==True):
        # Compute the mean of each block
        coarse_grained_array = reshaped_array.mean(axis=(1, 3))
    if (lsum==True):
        # Compute the mean of each block
        coarse_grained_array = reshaped_array.sum(axis=(1, 3))
    
    return coarse_grained_array

# Example usage
large_array = np.random.rand(3000, 3000)  # Replace with your actual array
block_size = 30
coarse_grained_array = coarse_grain(large_array, block_size)

print(coarse_grained_array.shape)  # Should output (100, 100)


## Set some paths and directories

In [ ]:
TopoRoot = '/glade/work/juliob/Topo/MyTopo/'
Case = 'ne120pg3_Sco25_Test03'
ogrid= 'ne120pg3'
Case = 'ne30pg3_Sco100_Test01'
ogrid= 'ne30pg3'
#Case = 'ne240pg3_Sco14_Test02'
#ogrid= 'ne240pg3'


In [ ]:
foo=glob.glob( TopoRoot +'/cases/'+ Case + '/output/topo*.nc')
FileN = foo[-1]
print(FileN)
topo=xr.open_dataset( FileN )
print(list(topo.variables))
print(topo.attrs)
raw=topo['terr_dev']+topo['terr_sm']
Loo = np.size(raw)
nc=int(np.sqrt( Loo/6 ))
print(int(nc))
raw=np.reshape( raw.values , (6,nc,nc) )
dev=np.reshape( topo['terr_dev'].values , (6,nc,nc) )
smoo=np.reshape( topo['terr_sm'].values , (6,nc,nc) )
tlats=np.reshape( topo['lat'].values , (6,nc,nc) )
tlons=np.reshape( topo['lon'].values , (6,nc,nc) )


In [ ]:
Po.sixpanel(aa=raw, clev=np.linspace(0,5000.,num=21) ,goofy=True , cmap='terrain' )
#Po.sixpanel(aa=yoog, clev=np.linspace(-1,2,num=21) )

## Read in 'remap' file. Currently an unformatted fortran file

In [ ]:
# RemapFile = 'remap_nc0540_Nsw008_Nrs000_Co012_Fi000_vX_20230106_1205.dat'
# FileN=TopoRoot +'/cases/'+ Case + '/output/'+RemapFile

# Do a file search in output directory
# Pick last (most recent) file
foo=glob.glob( TopoRoot +'/cases/'+ Case + '/output/remap*dat')
FileN = foo[-1]

f=FortranFile( FileN, 'r')
idims=f.read_record( '<i4' )

nc=idims[0]

uniqi=f.read_record( '<f8'   ).reshape( 6,nc,nc)
aniso=f.read_record( '<f8'   ).reshape( 6,nc,nc)
anglx=f.read_record( '<f8'   ).reshape( 6,nc,nc)
mxdis=f.read_record( '<f8'   ).reshape( 6,nc,nc)
hwdth=f.read_record( '<f8'   ).reshape( 6,nc,nc)
clngt=f.read_record( '<f8'   ).reshape( 6,nc,nc)
block=f.read_record( '<f8'   ).reshape( 6,nc,nc)
profi=f.read_record( '<f8'   ).reshape( 6,nc,nc)
nodes=f.read_record( '<f8'   ).reshape( 6,nc,nc)
wedge=f.read_record( '<f8'   ).reshape( 6,nc,nc)
nodos=f.read_record( '<f8'   ).reshape( 6,nc,nc)
wedgo=f.read_record( '<f8'   ).reshape( 6,nc,nc)



In [ ]:
#plt.figure(figsize=(20, 20))

#residual = dev - nodos #wedge

rdgana=nodes
residual = dev - rdgana
residualUB = dev - wedgo

clevels=np.linspace(-500.,3500.,num=21)
cmap=plt.cm.terrain

panel = 5
ipan= panel-1
xlims=(0,3000) #( 200,1200 )
ylims=(0,3000) #( 2000, 3000 )


fig,ax = plt.subplots(2,2,figsize=(30,30) )
ax=ax.flatten()

ax[0].contourf( dev[ipan,:,:]  ,levels=clevels,cmap=cmap )
ax[0].set_xlim( xlims )
ax[0].set_ylim( ylims )
ax[0].set_aspect('equal') #,'box')

ax[1].contourf( rdgana[ipan,:,:] ,levels=clevels,cmap=cmap)
ax[1].set_xlim(  xlims )
ax[1].set_ylim(  ylims )
ax[1].set_aspect('equal') #,'box')

ax[2].contourf( residual[ipan,:,:] ,levels=clevels,cmap=cmap)
ax[2].set_xlim( xlims )
ax[2].set_ylim( ylims )
ax[2].set_aspect('equal') #,'box')

ax[3].contourf( residualUB[ipan,:,:] ,levels=clevels,cmap=cmap)
ax[3].set_xlim( xlims )
ax[3].set_ylim( ylims )
ax[3].set_aspect('equal') #,'box')



In [ ]:
#residual = dev - nodos #wedge

dev30 = np.sqrt( coarse_grain( array= dev[ipan,:,:]**2, block_size=30 ) )

residual = dev - profi
resLB30 = np.sqrt( coarse_grain( array= residual[ipan,:,:]**2, block_size=30 ) )

residual = dev - wedgo
resUB30 = np.sqrt( coarse_grain( array= residual[ipan,:,:]**2, block_size=30 ) )

residual = dev - wedge
resGa30 = np.sqrt( coarse_grain( array= residual[ipan,:,:]**2, block_size=30 ) )

residual = dev - nodes
resGb30 = np.sqrt( coarse_grain( array= residual[ipan,:,:]**2, block_size=30 ) )


plt.scatter( dev30.flatten() , resUB30.flatten() , label='w/resp WEDGO')
plt.scatter( dev30.flatten() , resLB30.flatten() , label='w/resp PROFI')
plt.scatter( dev30.flatten() , resGa30.flatten() , label='w/resp WEDGE (ext)'  , marker='+')
plt.scatter( dev30.flatten() , resGb30.flatten() , label='w/resp NODES (ext)'  , marker='x')
plt.plot( [0,1100], [0,1100] )
plt.plot( [0,1100], [0,650] )
plt.legend()
    
#plt.colorbar()

In [ ]:
####
ipan=4
residual = dev - nodes

resGb30 = np.sqrt( coarse_grain( array= residual[ipan,:,:]**2, block_size=30 ) )

isomark = np.where( residual > 10. , 1. , 0. )
isomark30 = coarse_grain( array= isomark[ipan,:,:], block_size=30 ) 

fig,ax=plt.subplots(1,2, figsize=(21,8))
ax=ax.flatten()
c0=ax[0].contourf( isomark30 )
c1=ax[1].contourf( resGb30 )
plt.colorbar(c0)
plt.colorbar(c1)

In [ ]:
plt.plot( isomark30.flatten())